## Parte 1

tenemos un circuito que funciona con pulsos altos y bajos. Hay los siguientes módulos:

- Flip-flop: con prefijo % estan inicialmente apagados, si reciben un pulso alto lo ignoran, si reciben un pulso bajo cambian su estado a encendido y mandan un pulso alto, si están encendidos se apagan y envían un pulso bajo.

- Conjunction: con prefijo &, recuerdan el pulso más reciente de cada entrada, inicialmente recuerdan todo pulsos bajos, cuando reciben un pulso actualizan su memoria, luego si todo lo que recuerdan es un pulso alto envían un pulso bajo, en otro caso envían un pulso alto.

- Broadcast: hay un único módulo de broadcast, cuando recibe un pulso lo envía igual a todos los destinos.

- Botón: al pulsarlo envía un único pulso bajo al broadcast.

Los pulsos se procesan en el orden en que son enviados.

tras pulsar el botón mil veces cuantos pulsos altos y bajos se hyan enviado? La respuesta es pulsos altos por pulsos bajos.

In [1]:
def get_input(filename):
    """devuelve un diccionario:
       {nombre: [tipo, estado, conexiones]}"""
    componentes = {}
    with open(filename) as file:
        for line in file:
            name, cons = line.strip().split('->')
            is_flip = name.startswith('%')
            is_conj = name.startswith('&')
            cons = cons.split(',')
            
            if is_flip:
                componentes[name[1:].strip()] = ['flip', False, [c.strip() for c in cons]]
            elif is_conj:
                componentes[name[1:].strip()] = ['conj', {}, [c.strip() for c in cons]]
            else:
                componentes[name.strip()] = ['other', None, [c.strip() for c in cons]]
    
    return componentes

In [24]:
def init_conj(componentes):
    # inicializa todos los conjuntores a falso para cada conexión entrante
    for k0, v0 in componentes.items():
        if v0[0] == 'conj':
            for k1, v1 in componentes.items():
                cons = v1[2]
                if k0 in cons:
                    v0[1][k1] = False
    return componentes

In [25]:
def run_circuit(componentes):
    q = [('broadcaster', False, None)]
    
    low_counter = 0
    high_counter = 0
    
    while len(q):
        name, pulse, prev = q.pop(0)
        
        if pulse:
            high_counter += 1
        else:
            low_counter += 1
            
        if name not in componentes:
            continue
            
        tipo, state, cons = componentes[name]
        
        if tipo == 'flip':
            if not pulse:
                if not state:
                    componentes[name][1] = True
                    out = True
                elif state:
                    componentes[name][1] = False
                    out = False
                    
                for con in cons:
                    q.append((con, out, name))
        elif tipo == 'conj':
            state[prev] = pulse
            
            if all(state.values()):
                out = False
            else:
                out = True
                
            for con in cons:
                q.append((con, out, name))
        else:
            for con in cons:
                q.append((con, pulse, name))
    return low_counter, high_counter

In [26]:
def run(componentes, ntimes):
    cl = 0
    ch = 0
    init_conj(componentes)
    for i in range(ntimes):
        l, h = run_circuit(componentes)
        cl += l
        ch += h
        
    return cl * ch

In [27]:
componentes = get_input('test1.txt')
run(componentes, 1000)

32000000

In [29]:
componentes = get_input('input.txt')
run(componentes, 1000)

834323022

## Parte 2

la máquina tiene un único módulo llamado rx, la máquina se enciende cuando un pulso bajo se envía a rx. Cuál es el mínimo número de veces que hay que pulsar el botón para que llegue un único pulso bajo a rx?

In [55]:
import math
from functools import reduce

In [52]:
def run_circuit_rx(componentes, iteracion, origenes, ciclos):
    q = [('broadcaster', False, None)]
       
    while len(q) > 0:
        
        #vamos guardando la primera vez que se pone a true un origen de rs
        for c in q:
            if c[2] in origenes and c[1] and c[2] not in ciclos:
                ciclos[c[2]] = iteracion
                
        name, pulse, prev = q.pop(0)


        if name not in componentes:
            continue

        tipo, state, cons = componentes[name]

        if tipo == 'flip':
            if not pulse:
                if not state:
                    componentes[name][1] = True
                    out = True
                elif state:
                    componentes[name][1] = False
                    out = False

                for con in cons:
                    q.append((con, out, name))
        elif tipo == 'conj':
            state[prev] = pulse

            if all(state.values()):
                out = False
            else:
                out = True

            for con in cons:
                q.append((con, out, name))
        else:
            for con in cons:
                q.append((con, pulse, name))
    return ciclos

In [64]:
def run_rx(componentes, ntimes):
    componentes = init_conj(componentes)
    
    # (&bt, &dl, &fr, &rv) -> &rs -> rx
    origenes = ['bt', 'dl', 'fr', 'rv']
    ciclos = {}
    for i in range(1, ntimes):
        c = run_circuit_rx(componentes, i, origenes, ciclos)
        
    return ciclos

In [73]:
def lcm(arr):
    #a partir de python 3.9 lcm ya es una función y no hay que hacer esto
    l = reduce(lambda x, y: (x * y) // math.gcd(x, y), arr)
    return l

In [74]:
componentes = get_input('input.txt')
ciclos = run_rx(componentes, 10_000)
lcm(ciclos.values())

225386464601017